In [42]:
%load_ext autoreload
%autoreload 2

import os
from pathlib import Path
import shutil
import csv
import re
from collections import defaultdict
from datetime import datetime
import pandas as pd

import sys
sys.path.append("./missing_data")

from mpl_toolkits.basemap import Basemap
from supdata import get_files_from_folder, extract_dates_pattern_airmass_rgb_20200101_0000
from supdata import load_cyclones_track_noheader, compute_pixel_scale, inside_tile
from supdata import split_into_tiles_subfolders_and_track_dates

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [26]:
latcorners = [30, 48]
loncorners = [-7, 46]
basemap_obj = Basemap(
    projection='geos',
    rsphere=(6378137.0, 6356752.3142),
    resolution='i',
    area_thresh=10000.,
    lon_0=9.5,
    satellite_height=3.5785831E7
)
x_center, y_center = m(9.5, 0)
Xmin, Ymin, px_scale_x, px_scale_y = compute_pixel_scale(basemap_obj, latcorners, loncorners)

In [142]:
1/px_scale_y/1000 * 224, 1/px_scale_x/1000 * 224

(757.5205385725667, 804.8615703175508)

In [17]:
# Percorso alla cartella dei frame
input_dir = "../fromgcloud"  # Cambia questo percorso
output_dir = "../airmassRGB/supervised"  # Percorso per salvare i CSV
os.makedirs(output_dir, exist_ok=True)

filenames = get_files_from_folder(folder=input_dir, extension="png")
#print(f"Trovati {len(filenames)} files")

file_metadata = []
for fname in filenames:
    start_dt = extract_dates_pattern_airmass_rgb_20200101_0000(fname.name)
    file_metadata.append((fname, start_dt))

sorted_files = sorted(file_metadata, key=lambda x: x[1])  # Ordina per start_dt
#random_fnames =  [item[0] for item in file_metadata]
sorted_filenames = [item[0] for item in sorted_files]
print(f" Ci sono {len(sorted_filenames)} files.")

 Ci sono 117764 files.


In [11]:
sorted_filenames = sorted_filenames[:32]

In [18]:
sf = [(p, dt) for p, dt in zip(sorted_filenames, [f[1] for f in sorted_files])]
print(f"sorted filenames, len = {len(sf)}")
subfolder_info = split_into_tiles_subfolders_and_track_dates(sorted_filenames=sf, output_dir=output_dir)
print(f"Creati {len(subfolder_info)} tile-video in total.")

sorted filenames, len = 117764
num_subfolders: 7360
0 tilex 0 	 tiley 0 

part_index 7360
1 tilex 112 	 tiley 0 part_index 7360
2 tilex 224 	 tiley 0 part_index 7360
3 tilex 336 	 tiley 0 part_index 7360
4 tilex 448 	 tiley 0 part_index 7360
5 tilex 560 	 tiley 0 part_index 7360
6 tilex 672 	 tiley 0 part_index 7360
7 tilex 784 	 tiley 0 part_index 7360
8 tilex 896 	 tiley 0 part_index 7360
9 tilex 1008 	 tiley 0 part_index 7360
10 tilex 0 	 tiley 112 part_index 7360
11 tilex 112 	 tiley 112 part_index 7360
12 tilex 224 	 tiley 112 part_index 7360
13 tilex 336 	 tiley 112 part_index 7360
14 tilex 448 	 tiley 112 part_index 7360
15 tilex 560 	 tiley 112 part_index 7360
16 tilex 672 	 tiley 112 part_index 7360
17 tilex 784 	 tiley 112 part_index 7360
18 tilex 896 	 tiley 112 part_index 7360
19 tilex 1008 	 tiley 112 part_index 7360
Creati 147200 tile-video in total.


In [19]:
subfolder_info

[{'folder': '../airmassRGB/supervised/part1_0_0',
  'datetimes': [datetime.datetime(2011, 11, 1, 0, 0),
   datetime.datetime(2011, 11, 1, 0, 5),
   datetime.datetime(2011, 11, 1, 0, 10),
   datetime.datetime(2011, 11, 1, 0, 15),
   datetime.datetime(2011, 11, 1, 0, 20),
   datetime.datetime(2011, 11, 1, 0, 25),
   datetime.datetime(2011, 11, 1, 0, 30),
   datetime.datetime(2011, 11, 1, 0, 35),
   datetime.datetime(2011, 11, 1, 0, 40),
   datetime.datetime(2011, 11, 1, 0, 45),
   datetime.datetime(2011, 11, 1, 0, 50),
   datetime.datetime(2011, 11, 1, 0, 55),
   datetime.datetime(2011, 11, 1, 1, 0),
   datetime.datetime(2011, 11, 1, 1, 5),
   datetime.datetime(2011, 11, 1, 1, 10),
   datetime.datetime(2011, 11, 1, 1, 15)],
  'tile_x': 0,
  'tile_y': 0},
 {'folder': '../airmassRGB/supervised/part2_0_0',
  'datetimes': [datetime.datetime(2011, 11, 1, 1, 20),
   datetime.datetime(2011, 11, 1, 1, 25),
   datetime.datetime(2011, 11, 1, 1, 30),
   datetime.datetime(2011, 11, 1, 1, 35),
   dat

In [20]:
len(subfolder_info)

147200

In [15]:
# 2) Carico i ciclonici
tracks_file = "./TRACKS_CL7.dat"  
df_tracks = load_cyclones_track_noheader(tracks_file)

In [22]:
df_tracks['year'] = df_tracks['time'].dt.year
df_tracks['month']= df_tracks['time'].dt.month
df_tracks['day']  = df_tracks['time'].dt.day
df_tracks['hour'] = df_tracks['time'].dt.hour
cyc_groups = df_tracks.groupby(['year','month','day','hour'])

In [123]:
m = df_tracks.year == 2018 #.unique()
mset = df_tracks[m].month == 9
df_tracks[m&mset]

,id_cyc,lat,lon,time,year,month,day,hour
131456,00001618,8.676,43.439,2018-09-01 00:00:00,2018,9,1,0
131457,00001618,8.943,43.447,2018-09-01 01:00:00,2018,9,1,1
131458,00001618,9.247,43.476,2018-09-01 02:00:00,2018,9,1,2
131459,00001618,9.518,43.518,2018-09-01 03:00:00,2018,9,1,3
131460,00001618,9.719,43.569,2018-09-01 04:00:00,2018,9,1,4
...,...,...,...,...,...,...,...,...
131895,00001622,24.821,38.373,2018-09-30 19:00:00,2018,9,30,19
131896,00001622,24.910,38.392,2018-09-30 20:00:00,2018,9,30,20
131897,00001622,24.998,38.418,2018-09-30 21:00:00,2018,9,30,21
131898,00001622,25.069,38.464,2018-09-30 22:00:00,2018,9,30,22


In [81]:
subfolder_info

[{'folder': '../airmassRGB/supervised/part1_0_0',
  'datetimes': [datetime.datetime(2011, 11, 1, 0, 0),
   datetime.datetime(2011, 11, 1, 0, 5),
   datetime.datetime(2011, 11, 1, 0, 10),
   datetime.datetime(2011, 11, 1, 0, 15),
   datetime.datetime(2011, 11, 1, 0, 20),
   datetime.datetime(2011, 11, 1, 0, 25),
   datetime.datetime(2011, 11, 1, 0, 30),
   datetime.datetime(2011, 11, 1, 0, 35),
   datetime.datetime(2011, 11, 1, 0, 40),
   datetime.datetime(2011, 11, 1, 0, 45),
   datetime.datetime(2011, 11, 1, 0, 50),
   datetime.datetime(2011, 11, 1, 0, 55),
   datetime.datetime(2011, 11, 1, 1, 0),
   datetime.datetime(2011, 11, 1, 1, 5),
   datetime.datetime(2011, 11, 1, 1, 10),
   datetime.datetime(2011, 11, 1, 1, 15)],
  'tile_x': 0,
  'tile_y': 0},
 {'folder': '../airmassRGB/supervised/part2_0_0',
  'datetimes': [datetime.datetime(2011, 11, 1, 1, 20),
   datetime.datetime(2011, 11, 1, 1, 25),
   datetime.datetime(2011, 11, 1, 1, 30),
   datetime.datetime(2011, 11, 1, 1, 35),
   dat

In [82]:
results = []
for info in subfolder_info:
    folder_path = info['folder']
    tile_x = info['tile_x']
    tile_y = info['tile_y']
    dt_list = info['datetimes']

    found = False
    for dt_ in dt_list:
        if dt_ is None:
            continue
        k = (dt_.year, dt_.month, dt_.day, dt_.hour)

        if k in cyc_groups.indices:
            idxes = cyc_groups.indices[k]
            group_df = df_tracks.iloc[idxes]
            for row_ in group_df.itertuples(index=False):
                if inside_tile(row_.lat, row_.lon, tile_x, tile_y,
                                224, 224,
                                basemap_obj,
                                x_center, y_center,
                                px_per_km_x=px_scale_x, 
                                px_per_km_y=px_scale_y):
                    found = True
                    break
        if found:
            print(f"Trovato: {k} coordinate: {row_.lat, row_.lon}")
            break

    label = 1 if found else 0
    results.append({
        "path": folder_path,
        "start": 1,      # fissi 1 e 16 come "start" e "end"
        "end":   16,
        "label": label,
        "start_time": dt_list[0],
        "end_time": dt_list[-1]
    })

df_out = pd.DataFrame(results, columns=["path","start","end","label","start_time","end_time"])

Trovato: (2014, 11, 10, 13) coordinate: (6.705, 28.169)
Trovato: (2014, 11, 10, 13) coordinate: (6.705, 28.169)
Trovato: (2020, 3, 16, 0) coordinate: (0.087, 27.247)
Trovato: (2020, 3, 16, 0) coordinate: (0.087, 27.247)
Trovato: (2020, 3, 16, 1) coordinate: (0.323, 27.374)
Trovato: (2020, 3, 16, 3) coordinate: (0.73, 27.812)
Trovato: (2014, 11, 10, 1) coordinate: (0.083, 28.423)
Trovato: (2014, 11, 10, 1) coordinate: (0.083, 28.423)
Trovato: (2014, 11, 10, 2) coordinate: (0.507, 28.548)
Trovato: (2014, 11, 10, 3) coordinate: (0.951, 28.705)
Trovato: (2014, 11, 10, 5) coordinate: (1.742, 29.009)
Trovato: (2014, 11, 10, 6) coordinate: (2.134, 29.136)
Trovato: (2014, 11, 10, 7) coordinate: (2.652, 29.265)
Trovato: (2014, 11, 10, 9) coordinate: (4.202, 29.28)
Trovato: (2014, 11, 10, 10) coordinate: (4.946, 29.052)
Trovato: (2014, 11, 10, 11) coordinate: (5.593, 28.734)
Trovato: (2014, 11, 10, 13) coordinate: (6.705, 28.169)
Trovato: (2020, 3, 16, 0) coordinate: (0.087, 27.247)
Trovato: (20

In [104]:
df_out[df_out.label ==1]

,path,start,end,label,start_time,end_time
22395,../airmassRGB/supervised/part316_336_0,1,16,1,2014-11-10 11:55:00,2014-11-10 13:10:00
22396,../airmassRGB/supervised/part317_336_0,1,16,1,2014-11-10 13:15:00,2014-11-10 14:30:00
23716,../airmassRGB/supervised/part1637_336_0,1,16,1,2020-03-15 23:05:00,2020-03-16 00:20:00
23717,../airmassRGB/supervised/part1638_336_0,1,16,1,2020-03-16 00:25:00,2020-03-16 01:40:00
23718,../airmassRGB/supervised/part1639_336_0,1,16,1,2020-03-16 01:45:00,2020-03-16 03:00:00
...,...,...,...,...,...,...
139242,../airmassRGB/supervised/part6763_896_112,1,16,1,2020-12-25 23:05:00,2020-12-26 00:20:00
139243,../airmassRGB/supervised/part6764_896_112,1,16,1,2020-12-26 00:25:00,2020-12-26 01:40:00
139244,../airmassRGB/supervised/part6765_896_112,1,16,1,2020-12-26 01:45:00,2020-12-26 03:00:00
139245,../airmassRGB/supervised/part6766_896_112,1,16,1,2020-12-26 03:05:00,2020-12-26 04:20:00


In [84]:
df_out = df_out.rename(columns={'folder':'path'})

In [85]:
df_out.to_csv("./df_videotiles_date.csv", index=False)

# Guardo alcuni video da cancellare dal disco per risparmiare spazio

In [128]:
df_out[df_out['path'].str.contains(r"/part43\d+", regex=True)].label.sum()

0

In [129]:
df_filt = df_out[df_out['path'].apply(os.path.isdir)]
print(f"Numero di cartelle esistenti: {len(df_filt)}, righe totali precedenti: {df_out.shape[0]}")

Numero di cartelle esistenti: 126001, righe totali precedenti: 147200


In [130]:
df_filt[df_filt.label==1].shape[0], df_filt[df_filt.label==0].shape[0], round(df_filt.label.sum()/df_filt.shape[0], 3)

(1512, 124489, 0.012)

In [62]:
def balance_df_by_label(df, label_column="label", random_state=478562):
    """
    Bilancia un DataFrame mantenendo tutte le righe con label=1
    e selezionando (in modo random) lo stesso numero di righe con label=0.
    Restituisce un DataFrame mescolato con uguale quantità di 1 e 0.
    
    Parametri:
    - df: DataFrame originale
    - label_column: il nome della colonna che contiene la label (default: 'label')
    - random_state: per riproducibilità del campionamento casuale
    """
    # Seleziona tutte le righe con label=1
    df_pos = df[df[label_column] == 1]
    # Seleziona tutte le righe con label=0
    df_neg = df[df[label_column] == 0]

    # Numero di righe positive
    num_pos = len(df_pos)
    # Esegui un sample sulle righe negative pari a num_pos
    df_neg_sampled = df_neg.sample(n=num_pos, random_state=random_state)

    # Concatena e mescola
    df_balanced = pd.concat([df_pos, df_neg_sampled])
    df_balanced = df_balanced.sample(frac=1, random_state=random_state).reset_index(drop=True)

    return df_balanced

In [131]:
df_balanced = balance_df_by_label(df_filt)
print(len(df_balanced), df_balanced['label'].value_counts())

3024 label
1    1512
0    1512
Name: count, dtype: int64


In [137]:
df_balanced


,path,start,end,label,start_time,end_time
0,../airmassRGB/supervised/part6762_784_112,1,16,1,2020-12-25 21:45:00,2020-12-25 23:00:00
1,../airmassRGB/supervised/part6102_784_112,1,16,1,2020-11-19 04:50:00,2020-11-19 06:05:00
2,../airmassRGB/supervised/part6101_784_112,1,16,1,2020-11-19 03:30:00,2020-11-19 04:45:00
3,../airmassRGB/supervised/part3246_560_0,1,16,1,2020-06-13 11:20:00,2020-06-13 12:35:00
4,../airmassRGB/supervised/part1925_336_112,1,16,0,2020-04-01 00:05:00,2020-04-01 01:20:00
...,...,...,...,...,...,...
3019,../airmassRGB/supervised/part1653_560_0,1,16,1,2020-03-16 20:25:00,2020-03-16 21:40:00
3020,../airmassRGB/supervised/part1993_896_112,1,16,0,2020-04-04 18:45:00,2020-04-04 20:00:00
3021,../airmassRGB/supervised/part3207_448_112,1,16,1,2020-06-11 07:20:00,2020-06-11 08:35:00
3022,../airmassRGB/supervised/part2788_336_112,1,16,0,2020-05-18 23:05:00,2020-05-19 00:20:00


In [135]:
l = df_balanced.shape[0]
#l = 100
tr_p = int(l * 0.7)
val_p = int(l * 0.9)
print(f"ranges train e validation:  {tr_p} - {val_p}")
df_train = df_balanced[:tr_p]
df_val = df_balanced[tr_p:val_p]
df_test = df_balanced[val_p:]

ranges train e validation:  2116 - 2721


In [136]:
out_csv_label = os.path.join("./", "train_supervised.csv")
df_train.to_csv(out_csv_label, index=False)
out_csv_label = os.path.join("./", "val_supervised.csv")
df_val.to_csv(out_csv_label, index=False)
out_csv_label = os.path.join("./", "test_supervised.csv")
df_test.to_csv(out_csv_label, index=False)

### copia le immagini in sottocartelle .../partN

In [ ]:
def split_into_subfolders_and_track_dates(images, output_dir, num_frames=16):    
    subfolder_info = []
    num_total_files = len(images)
    num_subfolders = num_total_files // num_frames

    for i in range(num_subfolders):
        subfolder_name = f"part{i+1}"
        subfolder_path = os.path.join(output_dir, subfolder_name)
        os.makedirs(subfolder_path, exist_ok=True)

        start_idx = i * num_frames
        end_idx = start_idx + num_frames

        # Copia effettiva dei 16 frame nella sottocartella
        for idx, file in enumerate(images[start_idx:end_idx]):
            new_name = os.path.join(subfolder_path, f"img_{idx+1:05d}.png")
            shutil.copy(file, new_name)

        # Esempio: estrai la data dal frame centrale
        mid_idx = start_idx + num_frames // 2
        dt = extract_dates_pattern_airmass_rgb_20200101_0000(images[mid_idx].name)
        
        # Salviamo in una lista: cartella e data
        subfolder_info.append({"folder": subfolder_path, "date": dt})

    return subfolder_info

In [34]:
#subfolders = split_into_subfolders(sorted_filenames, output_dir)
subfolder_info = split_into_subfolders_and_track_dates(sorted_filenames, output_dir)

In [16]:
#subfolder_info

### Suddivide in train/test/val e scrive i CSV

In [28]:
output_dir

'E:/Medicanes_Data/airmassRGB'

In [30]:
from build_dataset import create_csv

In [33]:
create_csv(output_dir)

File CSV generati:
Train: E:/Medicanes_Data/airmassRGB\train.csv
Test: E:/Medicanes_Data/airmassRGB\test.csv
Validation: E:/Medicanes_Data/airmassRGB\val.csv


In [29]:
total = len(subfolders)
train_split = int(total * 0.7)
test_split = int(total * 0.99)

# per riscrivere i file csv
subfolders = sorted([os.path.join(output_dir, d) for d in os.listdir(output_dir) if os.path.isdir(os.path.join(output_dir, d))])


train_dirs = subfolders[:train_split]
test_dirs = subfolders[train_split:test_split]
val_dirs = subfolders[test_split:]

# Scrive nei file CSV con il formato richiesto
def write_to_csv(dirs, csv_file):
    with open(csv_file, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(["path", "start", "end"])  # Intestazione
        for dir_path in dirs:
            writer.writerow([dir_path, 1, 16])  # Riga nel formato richiesto

write_to_csv(train_dirs, train_csv)
write_to_csv(test_dirs, test_csv)
write_to_csv(val_dirs, val_csv)

print(f"File CSV generati:\nTrain: {train_csv}\nTest: {test_csv}\nValidation: {val_csv}")

File CSV generati:
Train: E:/Medicanes_Data/airmassRGB\train.csv
Test: E:/Medicanes_Data/airmassRGB\test.csv
Validation: E:/Medicanes_Data/airmassRGB\val.csv


# Per il dataset supervisionato

In [35]:
def create_supervised_csv_from_info(subfolder_info, medicane_csv, out_csv):
    import csv
    from datetime import datetime
    import pandas as pd

    # Carica gli intervalli di medicane
    intervals = load_medicane_intervals(medicane_csv)


    with open(out_csv, 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(["path","start","end","label"])

        for item in subfolder_info:
            folder_path = item["folder"]
            dt = item["date"]
            if dt is None:
                # Se non riesci a parsare la data, skip o label = -1
                continue
            label = 1 if is_in_medicane(dt, intervals) else 0
            writer.writerow([folder_path, 1, 16, label])

    print(f"Creato CSV supervisionato in: {out_csv}")



def load_medicane_intervals(medicane_csv):
    """
    Legge un file CSV con le date di inizio/fine dei Medicane.
    Esempio: col start_date, end_date in formato 'YYYY-MM-DD HH:MM'
    """
    
    intervals = []
    df = pd.read_csv(medicane_csv)
    for _, row in df.iterrows():
        start_dt = datetime.strptime(row['Start_Date'], "%Y-%m-%d")
        end_dt   = datetime.strptime(row['End_Date'],   "%Y-%m-%d")
        intervals.append((start_dt, end_dt))
    return intervals

def is_in_medicane(date_to_check, intervals):
    return any(start <= date_to_check <= end for (start, end) in intervals)


In [36]:
folder_root = output_dir      # cartella con subfolder part1, part2 ...
medicane_csv = "./medicane_validi.csv"    # le date dei medicane
out_csv = "./dataset.csv"
create_supervised_csv_from_info(subfolder_info, medicane_csv, out_csv)

Creato CSV supervisionato in: ./dataset.csv
